# `pymathutils.mesh` submodule tests

In [1]:
import sys
from pathlib import Path

nb_dir = Path().resolve()
tests_dir = (nb_dir / ".." / "src" / "python").resolve()
sys.path.insert(0, str(tests_dir))

In [1]:
from pymathutils.mesh import load_vf_samples_from_ply, write_vf_samples_to_ply, load_mesh_samples_from_ply

In [3]:
import pyvista as pv
from mesh_viewer import get_half_edge_vector_field
import numpy as np

def plot_hem(m):
    pts_V, vecs_V = get_half_edge_vector_field(m)

    pv_m = pv.PolyData(m.xyz_coord_V, faces=np.hstack([np.full((len(m.V_cycle_F), 1), 3), m.V_cycle_F]))
    
    plotter = pv.Plotter(notebook=True)
    plotter.add_mesh(pv_m, color="lightblue", show_edges=True)
    plotter.add_arrows(pts_V, vecs_V, mag=1.0, color="red")  # Add arrows
    plotter.show(jupyter_backend='trame')
    return plotter

ModuleNotFoundError: No module named 'vtk'

In [ ]:
from pymathutils.mesh.pyutils import HalfEdgeMesh
from pymathutils.mesh import load_vf_samples_from_ply, write_vf_samples_to_ply

m = HalfEdgeMesh.init_icososphere(
    num_refinements=2,
    compute_he_stuff=True,
)

write_vf_samples_to_ply(m.xyz_coord_V, m.V_cycle_F, "output/icososphere2_vf.ply")

m = HalfEdgeMesh.load_vf_ply("/home/wlough/projects/MeshBrane/data/example_ply/dumbbell_coarse_vf.ply")

write_vf_samples_to_ply(m.xyz_coord_V, m.V_cycle_F, "output/dumbbell_coarse_vf.ply")

plot_hem(m)


In [1]:
from pymathutils.mesh.pyutils import HalfEdgeMesh
from pymathutils.mesh import load_vf_samples_from_ply, write_vf_samples_to_ply, load_mesh_samples_from_ply, write_mesh_samples_to_ply

m = HalfEdgeMesh.init_icososphere(
    num_refinements=2,
    compute_he_stuff=True,
)

write_vf_samples_to_ply(m.xyz_coord_V, m.V_cycle_F, "output/icososphere2_vf.ply")

m = HalfEdgeMesh.load_vf_ply("/home/wlough/projects/MeshBrane/data/example_ply/dumbbell_coarse_vf.ply")

write_vf_samples_to_ply(m.xyz_coord_V, m.V_cycle_F, "output/dumbbell_coarse_vf.ply")




In [ ]:
write_vf_samples_to_ply(m.xyz_coord_V, m.V_cycle_F, "output/dumbbell_coarse_vf.ply", use_binary=False)

In [4]:
# load_mesh_samples_from_ply("/home/wlough/projects/MeshBrane/data/example_ply/dumbbell_coarse_vf.ply")
inply = "/home/wlough/projects/MeshBrane/data/example_ply/dumbbell_coarse_he.ply"
outply = "/home/wlough/projects/MathUtils/scratch/dumbbell_coarse_he.ply"
dat=load_mesh_samples_from_ply(inply)
write_mesh_samples_to_ply(dat, outply, use_binary=False)

In [3]:
!ls

findiff_tests.ipynb  __pycache__		       special_funs.py
mesh_tests.ipynb     random_tests.ipynb		       special_tests.ipynb
mesh_tests.py	     run_findiff_tests.py	       testing.py
plot.ipynb	     run_spherical_harmonics_tests.py
plot.py		     slider2d.ipynb
